<a href="https://colab.research.google.com/github/ericburdett/author-id-network/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Author Identification**
#### Eric Burdett

This notebook contains code that can successfully identify authors given a page of handwritten text. The datasets used come from the Missionary Journals dataset that is made available from the BYU library. This work is based off of the following papers:

*   Paper #1
*   Paper #2
